## Preprocessing of NZSeek data
data file name: NZ_Admin_JOBS.xlsx

In [75]:
import pandas as pd
import numpy as np

In [185]:
df = pd.read_excel('./data20211221/NZ_Admin_JOBS.xlsx')
df.head()

,字段1,字段1_link,字段2,字段3,字段4,字段5
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...


In [186]:
df.describe()

,字段1,字段1_link,字段2,字段3,字段4,字段5
count,2708,2708,2686,2708,2708,2708
unique,548,2708,475,147,54,93
top,Executive Assistant,https://www.seek.co.nz/job/50582301?type=promo...,Beyond Recruitment - Winner – Best Innovation ...,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...
freq,301,1,285,654,572,715


Replace columne names with more informative ones.

In [187]:
df.columns = ['job_title', 'source_link', 'employer', 'loc', 'post_time', 'job_type']
df.head()

,job_title,source_link,employer,loc,post_time,job_type
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...


In [188]:
import re

location_area_pattern = r'location:\s(?P<loc>(?:(?!area:)[\w\s])+)(area:\s(?P<area>[\w\s&]+))?'
location_area_pattern_for_split = r'location:\s(?:(?!area:)[\w\s])+(?:area:\s[\w\s&]+)?'

cls_subcls_pattern = r'classification:(?P<cls>[\w\s&,]+)subClassification:(?P<subcls>[\w\s&,]+)'
cls_subcls_pattern_for_split = r'classification:(?:[\w\s&,]+)subClassification:(?:[\w\s&,]+)'
def split_location_area(text):
    location, area = np.nan, np.nan
    text = text.strip()
    if len(text) == 0:
        return location, area
    match = re.search(location_area_pattern, text)
    if match is not None:
        location = match.group('loc')
        area = match.group('area')
    return location, area

#def split_location_area(text):
#    location, area = np.nan, np.nan
#    text = text.strip()
#    if len(text) == 0:
#        return location, area
#    sects = text.split(r'area:')
#    
#    match = re.search(r'location:(?P<loc>[\w\s&]+)', sects[0])
#    location = match.group('loc')
#    if len(sects) == 2:
#        
#        match = re.search(r'(?P<area>[\w\s&]+)', sects[1])
#        area = match.group('area')
#    return location, area
print("==========split_location_area=========")
print(split_location_area('location: AucklandAucklandarea: Auckland CentralAuckland Central'))
print(split_location_area('location: AucklandAucklandarea: Papakura & FranklinPapakura & Franklin'))
print(split_location_area('location: WaikatoWaikatoarea: HamiltonHamilton,Competitive hourly rates $$'))
print(split_location_area('location: WellingtonWellington'))
print(split_location_area('location: WellingtonWellington,Attractive remuneration'))

def remove_repetition(text):
    if text is None or text is np.nan:
        return text
    text = text.strip()
    if len(text) % 2 != 0:
        return text
    last_half_start = int(len(text) / 2)
    if text[:last_half_start] != text[last_half_start:]:
        return text
    return text[:last_half_start]

assert('Auckland' == remove_repetition(' AucklandAuckland'))
print("==========remove_repetition=========")
print(remove_repetition(' AucklandAuckland'))
print(remove_repetition(' Auckland CentralAuckland Central'))
print(remove_repetition(' Papakura & FranklinPapakura & Franklin'))
print(remove_repetition(' HamiltonHamilton'))
print(remove_repetition(' Hamilton'))

def extract_location_area(text):
    location, area = np.nan, np.nan
    if text is None or text is np.nan:
        return location, area
    text = text.strip()
    if len(text) == 0:
        return location, area
    location, area = split_location_area(text)
    location = remove_repetition(location)
    area = remove_repetition(area)
    return location, area
print("==========extract_location_area=========")
print(extract_location_area(' location: AucklandAucklandarea: Auckland CentralAuckland Central'))
print(extract_location_area('location: AucklandAucklandarea: Papakura & FranklinPapakura & Franklin'))
print(extract_location_area('location: WaikatoWaikatoarea: HamiltonHamilton,Competitive hourly rates $$'))
print(extract_location_area('location: WellingtonWellington'))
print(extract_location_area('location: WellingtonWellington,Attractive remuneration'))
print(extract_location_area('location: AucklandAucklandarea: Auckland CentralAuckland Central,Includes life and health insurance'))
print(extract_location_area('location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,$60 000 - $70 000 per annum'))
print(extract_location_area('location: AucklandAucklandarea: Waitakere & West AucklandWaitakere & West Auckland,$60 - $65k + Career Progression to QS / leadership'))


def split_cls_subcls(text):
    cls, subcls = np.nan, np.nan
    if text is None or text is np.nan:
        return cls, subcls
    text = text.strip()
    if len(text) == 0:
        return cls, subcls
    match = re.match(cls_subcls_pattern, text)
    if match is not None:
        cls = match.group('cls')
        subcls = match.group('subcls')
    return cls, subcls

print("==========split_cls_subcls=========")
print(split_cls_subcls("classification: Administration & Office SupportAdministration & Office SupportsubClassification: PA, EA & SecretarialPA, EA & Secretarial"))
print(split_cls_subcls("Competitive hourly rates $$"))

def extract_cls_subcls(text):
    cls, subcls = np.nan, np.nan
    if text is None or text is np.nan:
        return cls, subcls
    text = text.strip()
    if len(text) == 0:
        return cls, subcls
    cls, subcls = split_cls_subcls(text)
    cls = remove_repetition(cls)
    subcls = remove_repetition(subcls)
    return cls, subcls
    
print("==========extract_cls_subcls=========")    
print(extract_cls_subcls(text=np.nan))
print(extract_cls_subcls("classification: Administration & Office SupportAdministration & Office SupportsubClassification: PA, EA & SecretarialPA, EA & Secretarial"))
print(extract_cls_subcls("Competitive hourly rates $$"))


def collect_noises_from_loc_job_type(loc, job_type):
    noise_loc = ''
    noise_job_type = ''
    if loc is not None and loc is not np.nan and len(loc.strip()) > 0:
        loc = loc.strip()
        sects = re.split(location_area_pattern_for_split, loc)
        noise_loc = ' '.join(sects)
        noise_loc = noise_loc.strip().lstrip(',')
    if job_type is not None and job_type is not np.nan and len(job_type.strip()) > 0:
        job_type = job_type.strip()
        sects = re.split(cls_subcls_pattern_for_split, job_type)
        noise_job_type = ' '.join(sects)
        noise_job_type = noise_job_type.strip().lstrip(',')
    noise = noise_loc + ', ' + noise_job_type
    noise = noise.strip(',').strip()
    if len(noise) == 0:
        noise = np.nan
    return noise
    
    
print("==========collect_noises_from_loc_job_type=========")    
print(collect_noises_from_loc_job_type('location: WellingtonWellington,Attractive remuneration', "Competitive hourly rates $$"))
print(collect_noises_from_loc_job_type('location: WellingtonWellington,Attractive remuneration', "Competitive hourly rates $$"))
def collect_time_featured_private_from_post_time(text):
    post_time_ago, featured, private_advertiser = np.nan, np.nan, np.nan
    if text is not None and text is not np.nan and len(text.strip()) > 0:
        
        if re.search(r'featured', text, flags=re.IGNORECASE):
            featured = True
        if re.search(r'Private\s+Advertiser', text, flags=re.IGNORECASE):
            private_advertiser = True
        match = re.search(r'(?P<ptime>.+)\s+ago,\s?at', text)
        if match is not None:
            post_time_ago = match.group('ptime')
            post_time_ago = post_time_ago.strip()
    return post_time_ago, featured, private_advertiser

print("==========collect_time_featured_private_from_post_time=========")    
print(collect_time_featured_private_from_post_time("4d ago,at,Private Advertiser"))
print(collect_time_featured_private_from_post_time("Featured,at,Private Advertiser"))
print(collect_time_featured_private_from_post_time("7m ago,at"))
print(collect_time_featured_private_from_post_time("Featured,at"))


def represent_post_time_ago_in_hour(text):
    post_time_in_hour = np.nan
    if text is not None and text is not np.nan and len(text.strip()) > 0:
        text = text.strip()
        match = re.search(r'(?P<num>\d+)(?P<unit>\w)', text)
        if match is not None:
            num = match.group('num')
            num = int(num)
            unit = match.group('unit')
            unit = unit.lower()
            if unit == 'd':
                post_time_in_hour = num * 24
            elif unit == 'h':
                post_time_in_hour = num
            elif unit == 'm':
                post_time_in_hour = num / 60
    return post_time_in_hour

print("==========represent_post_time_ago_in_hour=========")    
print(represent_post_time_ago_in_hour(np.nan))
print(represent_post_time_ago_in_hour("4d"))
print(represent_post_time_ago_in_hour("40d"))
print(represent_post_time_ago_in_hour("7m"))
print(represent_post_time_ago_in_hour("5h"))

==========split_location_area=========
('AucklandAuckland', 'Auckland CentralAuckland Central')
('AucklandAuckland', 'Papakura & FranklinPapakura & Franklin')
('WaikatoWaikato', 'HamiltonHamilton')
('WellingtonWellington', None)
('WellingtonWellington', None)
==========remove_repetition=========
Auckland
Auckland Central
Papakura & Franklin
Hamilton
Hamilton
==========extract_location_area=========
('Auckland', 'Auckland Central')
('Auckland', 'Papakura & Franklin')
('Waikato', 'Hamilton')
('Wellington', None)
('Wellington', None)
('Auckland', 'Auckland Central')
('Bay of Plenty', 'Tauranga')
('Auckland', 'Waitakere & West Auckland')
==========split_cls_subcls=========
(' Administration & Office SupportAdministration & Office Support', ' PA, EA & SecretarialPA, EA & Secretarial')
(nan, nan)
==========extract_cls_subcls=========
(nan, nan)
('Administration & Office Support', 'PA, EA & Secretarial')
(nan, nan)
==========collect_noises_from_loc_job_type=========
Attractive remuneration, C

In [189]:
df['location'] = df['loc'].apply(lambda text: extract_location_area(text)[0])
df['area'] = df['loc'].apply(lambda text: extract_location_area(text)[1])
df['classification'] = df['job_type'].apply(lambda text: extract_cls_subcls(text)[0])
df['subClassification'] = df['job_type'].apply(lambda text: extract_cls_subcls(text)[1])
df['misc_info'] = df.apply(lambda row: collect_noises_from_loc_job_type(row['loc'], row['job_type']), axis=1)
df['post_time_ago'] = df['post_time'].apply(lambda text: collect_time_featured_private_from_post_time(text)[0])
df['is_featured'] = df['post_time'].apply(lambda text: collect_time_featured_private_from_post_time(text)[1])
df['is_private_advertiser'] = df['post_time'].apply(lambda text: collect_time_featured_private_from_post_time(text)[2])
df['post_time_ago_in_hour'] = df['post_time_ago'].apply(lambda text: represent_post_time_ago_in_hour(text))
df.head()

,job_title,source_link,employer,loc,post_time,job_type,location,area,classification,subClassification,misc_info,post_time_ago,is_featured,is_private_advertiser,post_time_ago_in_hour
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,Bay of Plenty,Tauranga,Administration & Office Support,Office Management,NaN,NaN,True,True,NaN
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...,Bay of Plenty,Tauranga,Administration & Office Support,Receptionists,NaN,NaN,True,NaN,NaN
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...,Auckland,None,Administration & Office Support,Other,NaN,4d,NaN,NaN,96.0
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...,Southland,Invercargill,Administration & Office Support,Administrative Assistants,NaN,1h,NaN,NaN,1.0
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...,Canterbury,Christchurch,Administration & Office Support,Client & Sales Administration,NaN,4d,NaN,True,96.0


In [191]:
df.describe(include='all')

,job_title,source_link,employer,loc,post_time,job_type,location,area,classification,subClassification,misc_info,post_time_ago,is_featured,is_private_advertiser,post_time_ago_in_hour
count,2708,2708,2686,2708,2708,2708,2708,2167,2105,2105,603,2688,20,22,2688.000000
unique,548,2708,475,147,54,93,15,38,1,9,84,38,1,1,NaN
top,Executive Assistant,https://www.seek.co.nz/job/50582301?type=promo...,Beyond Recruitment - Winner – Best Innovation ...,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...,Auckland,Auckland Central,Administration & Office Support,Administrative Assistants,"Competitive hourly rates $$, Competitive hourl...",27d,True,True,NaN
freq,301,1,285,654,572,715,891,751,2105,715,280,572,20,22,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501.603255
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.157844
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116667
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,624.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,648.000000


In [194]:
df_cleaned = df[['job_title', 
                'source_link', 
                'employer', 
                'location', 
                'area', 
                'post_time_ago', 
                'post_time_ago_in_hour', 
                'is_featured', 
                'is_private_advertiser',  
                'classification', 
                'subClassification', 
                'misc_info']]
df_cleaned.describe(include='all')

,job_title,source_link,employer,location,area,post_time_ago,post_time_ago_in_hour,is_featured,is_private_advertiser,classification,subClassification,misc_info
count,2708,2708,2686,2708,2167,2688,2688.000000,20,22,2105,2105,603
unique,548,2708,475,15,38,38,NaN,1,1,1,9,84
top,Executive Assistant,https://www.seek.co.nz/job/50582301?type=promo...,Beyond Recruitment - Winner – Best Innovation ...,Auckland,Auckland Central,27d,NaN,True,True,Administration & Office Support,Administrative Assistants,"Competitive hourly rates $$, Competitive hourl..."
freq,301,1,285,891,751,572,NaN,20,22,2105,715,280
mean,NaN,NaN,NaN,NaN,NaN,NaN,501.603255,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,205.157844,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.116667,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,336.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,624.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,648.000000,NaN,NaN,NaN,NaN,NaN


In [197]:
for col in df_cleaned.columns:
    print(col, df_cleaned[col].isna().sum())

job_title 0
source_link 0
employer 22
location 0
area 541
post_time_ago 20
post_time_ago_in_hour 20
is_featured 2688
is_private_advertiser 2686
classification 603
subClassification 603
misc_info 2105


In [198]:
df_cleaned['misc_info'].unique()

array([nan, '$20 per hour, $20 per hour',
       '$20 - $24.99 per hour, $20 - $24.99 per hour',
       '$19 to $22 per hour, $19 to $22 per hour',
       '$40,000 - $49,999, $40,000 - $49,999',
       '$25 per hour, $25 per hour',
       'Includes life and health insurance, Includes life and health insurance',
       '$60 000 - $70 000 per annum, $60 000 - $70 000 per annum',
       '$20.00 per hour, $20.00 per hour',
       '$55,000 - $69,999, $55,000 - $69,999',
       '$20.00 per hour., $20.00 per hour.', '$21 per hour, $21 per hour',
       '$55,000 - $64,999, $55,000 - $64,999',
       '$50,000 - $55,000 plus benefits, $50,000 - $55,000 plus benefits',
       'incl Life & Health Insurance, incl Life & Health Insurance',
       '$45,000 - $49,999, $45,000 - $49,999',
       '$55,000 - $60,000 per annum pro rata, $55,000 - $60,000 per annum pro rata',
       '$48,835 - $63,221, $48,835 - $63,221',
       '$20 - $29.99 per hour, $20 - $29.99 per hour', '$24-$28, $24-$28',
       'Up

In [199]:
df_cleaned['location'].unique()

array(['Bay of Plenty', 'Auckland', 'Southland', 'Canterbury',
       'Northland', 'Wellington', 'West Coast', 'Tasman', 'Manawatu',
       'Waikato', 'Hawkes Bay', 'Otago', 'Taranaki', 'Marlborough',
       'Gisborne'], dtype=object)

In [200]:
df_cleaned['area'].unique()

array(['Tauranga', None, 'Invercargill', 'Christchurch', 'Whangarei',
       'Wellington Central', 'Rotorua', 'Rest of West Coast', 'Nelson',
       'Manukau & East Auckland', 'Rest of Canterbury',
       'Timaru & South Canterbury', 'Auckland Central',
       'Rest of Manawatu', 'Papakura & Franklin', 'Rodney & North Shore',
       'Rest of Waikato', 'Hastings', 'Queenstown & Wanaka',
       'Rest of Otago', 'Hamilton', 'Hutt Valley', 'New Plymouth',
       'Palmerston North', 'Waitakere & West Auckland',
       'Rest of Taranaki', 'Dunedin', 'Rest of Bay of Plenty', 'Blenheim',
       'Thames & Coromandel', 'Rest of Tasman', 'Napier',
       'Rest of Hawkes Bay', 'Gisborne', 'Rest of Northland',
       'Porirua & Kapiti Coast', 'Rest of Wellington Region', 'Wanganui',
       'Taupo'], dtype=object)

In [201]:
df_cleaned['classification'].unique()

array(['Administration & Office Support', nan], dtype=object)

In [202]:
df_cleaned['subClassification'].unique()

array(['Office Management', 'Receptionists', 'Other',
       'Administrative Assistants', 'Client & Sales Administration',
       'Data Entry & Word Processing', nan, 'PA, EA & Secretarial',
       'Contracts Administration',
       'Records Management & Document Control'], dtype=object)

In [203]:
df_cleaned.to_csv('data20211221/NZ_Admin_JOBS_cleaned.csv')